In [2]:
import numpy
import pandas
import matplotlib.pyplot
import csv
import torch
import math
from sklearn.model_selection import train_test_split

https://jamesmccaffrey.wordpress.com/2023/09/15/can-a-neural-network-predict-poisson-distributed-target-data/

In [3]:
schools = pandas.read_csv("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/School_Counts/ceara_school_count_data.csv")

schools = schools.iloc[:, 3:]

schools = schools.dropna()

df = schools

df.head()

,NUMPOINTS,ID,land,pop,built_s,built_v,smod,x,y
0,0,1.0,1000000.0,0.000000,0,0,11,-4.146928e+06,-415647.743962
1,0,2.0,1000000.0,0.000000,0,0,11,-4.146995e+06,-416168.514849
2,0,3.0,1000000.0,89.137142,4644,11618,12,-4.145898e+06,-412699.018444
3,0,4.0,1000000.0,8.601842,359,898,11,-4.146006e+06,-413307.519153
4,0,5.0,1000000.0,2.825484,141,353,11,-4.146103e+06,-414290.948276


In [4]:
# split out the validation set
train_x, val_x, train_y, val_y,train_evy,val_evy = train_test_split(
    df.drop(labels = ['NUMPOINTS','pop'],axis = 1),
    df['NUMPOINTS'],
    df['pop'],
    test_size = 0.15,
    random_state = 0,
    shuffle = True
)

# now split train and test
train_x, test_x, train_y, test_y,train_evy,test_evy = train_test_split(
    train_x,
    train_y,
    train_evy,
    test_size = 0.2,
    random_state = 0,
    shuffle = True
)

mapper_land = {j:str(int(j)) for j in df['land'].unique()}
mapper_smod = {j:str(int(j)) for j in df['smod'].unique()}


# helper function
def transform_data(data):
    
    '''
    Function to apply transformations to input data.
    '''
    
    d = data.copy()
    
    d['land'] = d['land'].map(mapper_land).astype(str)
    d['smod'] = d['smod'].map(mapper_smod).astype(str)
    
    return d

# apply groupings and transformations
train_x = transform_data(train_x)

train_x.head()

,ID,land,built_s,built_v,smod,x,y
118929,118930.0,903084,0,0,11,-3.881370e+06,-945241.256537
33969,33970.0,744911,1431,3579,11,-4.038370e+06,-731241.256537
84078,84079.0,1000000,1812,4533,11,-3.946370e+06,-718241.256537
3072,3073.0,1000000,0,0,11,-4.117370e+06,-575241.256537
145099,145100.0,999899,15215,38043,12,-3.805370e+06,-627241.256537


In [19]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.hid1 = torch.nn.Linear(4, 10)  # 4-(10-10)-1
    self.hid2 = torch.nn.Linear(10, 10)
    self.oupt = torch.nn.Linear(10, 1)

    torch.nn.init.xavier_uniform_(self.hid1.weight)
    torch.nn.init.zeros_(self.hid1.bias)
    torch.nn.init.xavier_uniform_(self.hid2.weight)
    torch.nn.init.zeros_(self.hid2.bias)
    torch.nn.init.xavier_uniform_(self.oupt.weight)
    torch.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = torch.tanh(self.hid1(x))
    z = torch.tanh(self.hid2(z))
    z = self.oupt(z)  # no activation
    return z

In [20]:
def accuracy(model, ds):
  # assumes model.eval()
  n_correct = 0; n_wrong = 0

  for i in range(len(ds)):
    X = ds[i][0]   # 2-d
    Y = ds[i][1]   # 2-d      # target y as float
    target_y = torch.round(Y)       # as int
    with torch.no_grad():
      raw_y = model(X)     # predicted y as float
    pred_y = torch.round(raw_y)  # as int

    if pred_y == target_y:
      n_correct += 1
    else:
      n_wrong += 1
  acc = (n_correct * 1.0) / (n_correct + n_wrong)
  return acc

In [21]:
def train(model, ds, bs, lr, me, le):
  # dataset, bat_size, lrn_rate, max_epochs, log interval
  train_ldr = torch.utils.data.DataLoader(ds, batch_size=bs,
    shuffle=True)
  loss_func = torch.nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  for epoch in range(0, me):
    epoch_loss = 0.0  # for one full epoch
    for (b_idx, batch) in enumerate(train_ldr):
      X = batch[0]  # predictors
      y = batch[1]  # target income
      optimizer.zero_grad()
      oupt = model(X)
      loss_val = loss_func(oupt, y)  # a tensor
      epoch_loss += loss_val.item()  # accumulate
      loss_val.backward()  # compute gradients
      optimizer.step()     # update weights

    if epoch % le == 0:
      print("epoch = %4d  |  loss = %0.4f" % \
        (epoch, epoch_loss)) 

In [23]:
net = Net()#.to(device)